In [90]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests
import numpy as np
import pandas as pd

In [133]:
def pars_hh(link, D):
    headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    req = requests.get(link, headers=headers)
    html = bs(req.text, 'lxml')
    vacancy_list = html.findAll('div', {'data-qa':['vacancy-serp__vacancy vacancy-serp__vacancy_premium', 'vacancy-serp__vacancy']})
    for vacancy in vacancy_list:
        info = vacancy.find('div', {'class':'resume-search-item__name'}).findChild().findChild()
        name = info.getText()
        link = info['href']
        compensation = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        if compensation:
            spl_comp = compensation.split(' ')
            if 'от' in spl_comp:
                spl_comp.remove('от')
            if 'до' in spl_comp:
                spl_comp.remove('до')
            currency = spl_comp[1]
            real_compensation = spl_comp[0]
            if '-' in real_compensation:
                min_comp = real_compensation.split('-')[0]
                max_comp = real_compensation.split('-')[1]
            else:
                min_comp = max_comp = real_compensation
        else:
            min_comp = max_comp = currency = np.NaN
        D['name'].append(name)
        D['min'].append(min_comp)
        D['max'].append(max_comp)
        D['currency'].append(currency)
        D['link'].append(link)
        D['site'].append('hh.ru')
        
    return D

def pars_sj(link, D):
    headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    req = requests.get(link_sj, headers=headers)
    html = bs(req.text, 'lxml')
    vacancy_list = html.findAll('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})
    for vacancy in vacancy_list:
        info = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText()
        salary = vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
        if not salary:
            min_comp = np.NaN
            max_comp = np.NaN
            currency = np.NaN
        if 'По' in salary:
            min_comp = salary
            max_comp = salary
            currency = np.NaN
        else:
            currency = salary[-1]
            salary = salary[:-1]
            if '—' in salary:
                min_comp = salary.split('—')[0]
                max_comp = salary.split('—')[1]
            else:
                min_comp = salary
                max_comp = salary
        link = main_link_sj + vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).findParent()['href']
        D['name'].append(info)
        D['min'].append(min_comp)
        D['max'].append(max_comp)
        D['currency'].append(currency)
        D['link'].append(link)
        D['site'].append('superjob.ru')
        
    return D

In [112]:
main_link_sj = 'https://www.superjob.ru'
spec_link_sj = '/vacancy/search/?keywords='+spec
link_sj = main_link_sj + spec_link_sj
headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
req = requests.get(link_sj, headers=headers)
html = bs(req.text, 'lxml')
vacancy_list = html.findAll('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})


In [136]:
D = {'name':[], 'min':[], 'max':[], 'currency':[], 'link':[], 'site':[]}
n = int(input('Сколько страниц парсить?\t'))
main_link = 'https://hh.ru/search/vacancy'
spec = input('Введите должность\t')
spec_link = '?text=' + spec
link = main_link + spec_link
main_link_sj = 'https://www.superjob.ru'
spec_link_sj = '/vacancy/search/?keywords='+spec
link_sj = main_link_sj + spec_link_sj
i = 1
while i <= n:
    D = pars_hh(link, D)
    D = pars_sj(link_sj, D)
    link += '&page='+str(i)
    link_sj += '&page='+str(i+1)
    i += 1  
pd.DataFrame(D)

Сколько страниц парсить?	3
Введите должность	python


,name,min,max,currency,link,site
0,Python Developer,150 000,200 000,руб.,https://ulyanovsk.hh.ru/vacancy/34212917?query...,hh.ru
1,Middle Python Developer,NaN,NaN,NaN,https://ulyanovsk.hh.ru/vacancy/32858600?query...,hh.ru
2,Python Engineer,1 000,2 000,USD,https://ulyanovsk.hh.ru/vacancy/34128405?query...,hh.ru
3,Middle Python Developer,NaN,NaN,NaN,https://ulyanovsk.hh.ru/vacancy/34187170?query...,hh.ru
4,Программист Python,500,1 500,USD,https://ulyanovsk.hh.ru/vacancy/34145172?query...,hh.ru
5,QA Engineer (Тестировщик ПО),120 000,120 000,руб.,https://ulyanovsk.hh.ru/vacancy/33925503?query...,hh.ru
6,Senior C++ Developer (Videostreaming),180 000,180 000,руб.,https://ulyanovsk.hh.ru/vacancy/31917536?query...,hh.ru
7,Front-End React.js Engineer,600,2 000,USD,https://ulyanovsk.hh.ru/vacancy/34128413?query...,hh.ru
8,Go/Python developer,NaN,NaN,NaN,https://ulyanovsk.hh.ru/vacancy/34123079?query...,hh.ru
9,Стажер Sberseasons Аналитик,NaN,NaN,NaN,https://ulyanovsk.hh.ru/vacancy/33421599?query...,hh.ru
